<a href="https://colab.research.google.com/github/Achillesy/CL-DETECTION2023/blob/master/UNet_5Fold_Keras_256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023 CL-DETECTION
[Cephalometric Landmark Detection in Lateral X-ray Images 2023](https://cl-detection2023.grand-challenge.org/)

## Prepare paths of input images and landmark distribution masks

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
curpath = os.getcwd()
workpath = curpath.split("Workspace")[0]
gData = os.path.join(workpath, "Workspace", "DataSet")

In [ ]:
root_dir = os.path.join(gData, "grandchallenge", "cl_detection2023")
out_dir = os.path.join(os.getcwd(), "out")
print(root_dir)

D:\Xuchu_Liu\Workspace\DataSet\grandchallenge\cl_detection2023


## What does one input image and mask look like?

In [ ]:
from types import SimpleNamespace

cfg = SimpleNamespace(**{})

cfg.img_size = 256
cfg.num_classes = 1

cfg.epochs = 100
cfg.batch_size = 4

cfg.fold = 0
cfg.seed = 20230803

cfg.mark_csv = os.path.join(root_dir, "labels", "resized_256.csv")
cfg.img_dir = os.path.join(root_dir, "resized_256_images")
cfg.train_csv = os.path.join(root_dir, "labels", "train_gt.csv")

In [ ]:
mark_df = pd.read_csv(cfg.mark_csv)
mark_df.head()

,FileName,Z,Fold,ABC,scale,X1,X2,X3,X4,X5,...,Y31,Y32,Y33,Y34,Y35,Y36,Y37,Y38,croppedW,croppedH
0,train_stack_000.jpg,1,0,2,0.1,110,195,174,82,177,...,216,246,251,95,122,246,190,197,1936,2400
1,train_stack_001.jpg,2,1,1,0.1,101,177,165,82,190,...,181,206,210,86,103,221,167,173,1936,2400
2,train_stack_002.jpg,3,2,1,0.1,115,199,182,82,200,...,202,227,231,94,103,246,175,184,1936,2400
3,train_stack_003.jpg,4,3,1,0.1,93,173,165,70,186,...,190,217,219,81,95,217,168,172,1936,2400
4,train_stack_004.jpg,5,4,1,0.1,110,185,166,77,175,...,202,230,236,96,112,221,177,187,1936,2400


In [ ]:
train_df = pd.read_csv(cfg.train_csv)
orig_df = train_df[train_df["Fold"] == cfg.fold].reset_index(drop=True)
orig_df.head()

,FileName,Z,Fold,ABC,scale,X1,X2,X3,X4,X5,...,Y29,Y30,Y31,Y32,Y33,Y34,Y35,Y36,Y37,Y38
0,train_stack_000.jpg,1,0,2,0.1,835,1473,1318,623,1342,...,1388,1718,2028,2304,2356,892,1146,2309,1777,1848
1,train_stack_005.jpg,6,0,1,0.1,833,1452,1349,618,1394,...,1426,1582,1844,2185,2271,747,892,2328,1668,1724
2,train_stack_010.jpg,11,0,2,0.1,861,1515,1391,637,1478,...,1453,1598,1816,2185,2226,879,1003,2228,1708,1760
3,train_stack_015.jpg,16,0,3,0.1,777,1352,1271,596,1329,...,1425,1549,1746,1962,2021,819,1026,2128,1583,1644
4,train_stack_020.jpg,21,0,1,0.1,855,1465,1370,636,1447,...,1362,1590,1814,2051,2082,878,1054,2140,1621,1697


## Prepare U-Net model

In [ ]:
import tensorflow as tf
from tensorflow import keras

np.random.seed(cfg.seed)
tf.random.set_seed(cfg.seed)

In [ ]:
def conv_block(input, num_filters):
    x = keras.layers.Conv2D(num_filters, 3, padding="same")(input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    x = keras.layers.Conv2D(num_filters, 3, padding="same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = keras.layers.MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(
        input
    )
    x = keras.layers.Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(image_size, num_classes):
    inputs = keras.Input(shape=(image_size, image_size, 3))
    f = [16, 32, 64, 128, 256]

    s1, p1 = encoder_block(inputs, f[0])
    s2, p2 = encoder_block(p1, f[1])
    s3, p3 = encoder_block(p2, f[2])
    s4, p4 = encoder_block(p3, f[3])

    b1 = conv_block(p4, f[4])

    d1 = decoder_block(b1, s4, f[3])
    d2 = decoder_block(d1, s3, f[2])
    d3 = decoder_block(d2, s2, f[1])
    d4 = decoder_block(d3, s1, f[0])

    outputs = keras.layers.Conv2D(
        num_classes, 1, padding="same", activation="sigmoid"
    )(d4)

    model = keras.models.Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
model = build_unet(cfg.img_size, cfg.num_classes)
# model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

## Divide training, validation, and test datasets

In [ ]:
list_df = mark_df[mark_df["Fold"] != cfg.fold].reset_index(drop=True)
test_df = mark_df[mark_df["Fold"] == cfg.fold].reset_index(drop=True)

train_len = int(0.8 * len(list_df))
train_df = list_df[:train_len].sample(frac=1).reset_index(drop=True)
valid_df = list_df[train_len:].reset_index(drop=True)

## Prepare data generation class

In [ ]:
import cv2
from functools import lru_cache

class DataGen(keras.utils.Sequence):
    def __init__(self, data_df, batch_size=8):
        self.data_df = data_df
        self.batch_size = batch_size
        self.length = len(self.data_df)
        self.gaussian_matrix = np.load("matrix_512.npy") / 255.
        self.on_epoch_end()

    @lru_cache(maxsize=None)  # Cache all results
    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        equalized_image = clahe.apply(gray_image)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        eroded_image = cv2.erode(equalized_image, kernel, iterations=1)
        dilated_image = cv2.dilate(eroded_image, kernel, iterations=1)
        denoised_image = cv2.GaussianBlur(dilated_image, (5, 5), 0)

        normalized_image = cv2.normalize(
            denoised_image,
            None,
            alpha=0,
            beta=1,
            norm_type=cv2.NORM_MINMAX,
            dtype=cv2.CV_32F,
        )
        output_image = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)
        return output_image

    def __load__(self, id_name):
        data = self.data_df.iloc[id_name]
        i_name = os.path.join(cfg.img_dir, data["FileName"])
        i_img = self.preprocess_image(i_name)

        i_lab = np.zeros((cfg.img_size, cfg.img_size, cfg.num_classes))
        resized_x_value = data[f"X{cfg.name+1}"]
        resized_y_value = data[f"Y{cfg.name+1}"]
        overlap_x = cfg.img_size - resized_x_value - 1
        overlap_y = cfg.img_size - resized_y_value - 1
        overlap_img = self.gaussian_matrix[
            overlap_y : overlap_y + cfg.img_size,
            overlap_x : overlap_x + cfg.img_size,
        ]
        i_lab[..., 0] = overlap_img

        o_img = tf.image.convert_image_dtype(i_img, tf.float32)
        o_lab = tf.image.convert_image_dtype(i_lab, tf.float32)
        return o_img, o_lab

    def __getitem__(self, index):
        image = []
        mask = []

        for id_name in range(index * self.batch_size, (index + 1) * self.batch_size):
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)

        image = np.array(image)
        mask = np.array(mask)

        return image, mask

    def on_epoch_end(self):
        pass

    def __len__(self):
        return self.length // self.batch_size

## Train the model

In [ ]:
train_gen = DataGen(train_df, batch_size=cfg.batch_size)
valid_gen = DataGen(valid_df, batch_size=cfg.batch_size)

train_steps = train_gen.__len__()
valid_steps = valid_gen.__len__()

In [ ]:
def custom_loss(y_true, y_pred):
    y_true_max_pos = tf.argmax(
        tf.reshape(y_true, (-1, cfg.img_size * cfg.img_size, cfg.num_classes)), axis=1
    )
    y_pred_max_pos = tf.argmax(
        tf.reshape(y_pred, (-1, cfg.img_size * cfg.img_size, cfg.num_classes)), axis=1
    )

    x_true_pos = y_true_max_pos // cfg.img_size
    x_true_pos = tf.cast(x_true_pos, tf.float32)
    y_true_pos = y_true_max_pos % cfg.img_size
    y_true_pos = tf.cast(y_true_pos, tf.float32)

    x_pred_pos = y_pred_max_pos // cfg.img_size
    x_pred_pos = tf.cast(x_pred_pos, tf.float32)
    y_pred_pos = y_pred_max_pos % cfg.img_size
    y_pred_pos = tf.cast(y_pred_pos, tf.float32)

    distance_loss = tf.reduce_mean(
        tf.sqrt(tf.square(x_true_pos - x_pred_pos) + tf.square(y_true_pos - y_pred_pos))
    )
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    pareto_loss = tf.stack([mse_loss, distance_loss], axis=0)

    return pareto_loss

In [ ]:
import time

train_start = time.time()

In [ ]:
cfg.name = 0 # 0~37
cfg.patience = 5
cfg.hdf5_name = f"UNet_5Fold{cfg.fold}_{cfg.img_size}_name{cfg.name+1:02d}.hdf5"

# model.load_weights("UNet_5Fold0_256_base.hdf5")
model.compile(optimizer="Nadam", loss="mse")

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    cfg.hdf5_name,
    monitor="val_loss",
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
)

earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=cfg.patience)

history = model.fit(
    train_gen,
    validation_data=valid_gen,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=cfg.epochs,
    verbose=1,
    callbacks=[checkpoint],
)
print(history.history.keys())

Epoch 1/100
64/64 [==============================] - 15s 98ms/step - loss: 0.0925 - val_loss: 0.1816
Epoch 2/100
64/64 [==============================] - 7s 116ms/step - loss: 0.0370 - val_loss: 0.1055
Epoch 3/100
64/64 [==============================] - 4s 64ms/step - loss: 0.0208 - val_loss: 0.0512
Epoch 4/100
64/64 [==============================] - 4s 63ms/step - loss: 0.0134 - val_loss: 0.0259
Epoch 5/100
64/64 [==============================] - 4s 65ms/step - loss: 0.0092 - val_loss: 0.0167
Epoch 6/100
64/64 [==============================] - 4s 62ms/step - loss: 0.0066 - val_loss: 0.0128
Epoch 7/100
64/64 [==============================] - 4s 63ms/step - loss: 0.0048 - val_loss: 0.0070
Epoch 8/100
64/64 [==============================] - 7s 104ms/step - loss: 0.0036 - val_loss: 0.0046
Epoch 9/100
64/64 [==============================] - 7s 105ms/step - loss: 0.0028 - val_loss: 0.0032
Epoch 10/100
64/64 [==============================] - 6s 101ms/step - loss: 0.0023 - val_loss: 0

In [ ]:
train_time = time.time() - train_start
hours = round(train_time // 3600)
minutes = round((train_time - hours * 3600) // 60)
seconds = round(train_time - hours * 3600 - minutes * 60)
print(
    f"Fold {cfg.fold} train completed, total time: {hours}:{minutes:02d}:{seconds:02d}"
)

In [ ]:
# summarize history for loss
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "valid"], loc="upper right")
plt.show()

## Visualize test predictions

In [ ]:
model.load_weights(cfg.hdf5_name)

In [ ]:
def preprocess(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(gray_image)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    eroded_image = cv2.erode(equalized_image, kernel, iterations=1)
    dilated_image = cv2.dilate(eroded_image, kernel, iterations=1)
    denoised_image = cv2.GaussianBlur(dilated_image, (5, 5), 0)

    normalized_image = cv2.normalize(
        denoised_image,
        None,
        alpha=0,
        beta=1,
        norm_type=cv2.NORM_MINMAX,
        dtype=cv2.CV_32F,
    )
    output_image = cv2.cvtColor(normalized_image, cv2.COLOR_GRAY2RGB)
    return output_image

In [ ]:
# Test dataset for prediction
total_err = 0
total_conf = 0
sdr_landmarks = 0
length = len(test_df)

for idx in range(length):
    data = test_df.iloc[idx]
    i_name = os.path.join(cfg.img_dir, data["FileName"])
    jpg = preprocess(i_name)

    v_jpg = tf.expand_dims(jpg, 0)
    pred = model.predict(v_jpg)
    pred = tf.squeeze(pred, axis=0)

    max_value_position = np.unravel_index(np.argmax(pred), pred.shape)
    pred_x_value = max_value_position[1]  # Store the x value
    pred_y_value = max_value_position[0]  # Store the y value0
    pred_conf = np.max(pred)  # Store the confidence

    resized_x_value = data[f"X{cfg.name+1}"]
    resized_y_value = data[f"Y{cfg.name+1}"]

    plt.subplots(1, 2, figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(jpg, "bone")
    plt.scatter(resized_x_value, resized_y_value, color="g", marker=".")
    plt.text(resized_x_value, resized_y_value, f"{cfg.name+1}", color="w")
    plt.scatter(pred_x_value, pred_y_value, color="r", marker=".")
    plt.plot(
        [resized_x_value, pred_x_value],
        [resized_y_value, pred_y_value],
        color="m",
    )
    plt.title(f'{data["FileName"]}')

    pred_x_value = np.round(pred_x_value * data["croppedW"] / cfg.img_size)
    pred_y_value = np.round(pred_y_value * data["croppedH"] / cfg.img_size)

    orig = orig_df.iloc[idx]
    orig_x_value = orig[f"X{cfg.name+1}"]
    orig_y_value = orig[f"Y{cfg.name+1}"]

    err_square = (orig_x_value - pred_x_value) ** 2 + (
        orig_y_value - pred_y_value
    ) ** 2
    err_float = np.array(err_square, dtype=float)
    err_val = np.sqrt(err_float) * data["scale"]
    plt.subplot(1, 2, 2)
    plt.imshow(pred, "bone")
    plt.title(f"Overlap Image err={err_val:.2f}mm conf={pred_conf*100:.2f}%")
    plt.show()

    sdr_landmarks += np.sum(err_val < 2)
    total_err += err_val
    total_conf += pred_conf

err = np.sum(total_err) / len(test_df) / cfg.num_classes
sdr = sdr_landmarks / len(test_df) / cfg.num_classes
print(f"err={err:.3f}mm\t sdr={sdr*100:.2f}%")

In [ ]:
pred.shape